<a href="https://colab.research.google.com/github/sk9623ck/Turerz-Sole-Propeitorship/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pdfplumber
import pandas as pd
import re
from datetime import datetime
import os

In [2]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.7 MB/s eta 0:00:00


In [16]:
class DocumentProcessor:
    def __init__(self, pdf_path):
        # Changed the path to a generic filename. User needs to upload 'Data Input.pdf' to Colab.
        self.pdf_path = pdf_path # Use the path passed during initialization
        self.text = ""
        self.extracted_data = []

    def extract_text_from_pdf(self):
        """Extract all text from the PDF file"""
        try:
            with pdfplumber.open(self.pdf_path) as pdf:
                for page in pdf.pages:
                    self.text += page.extract_text() + "\n"
            return True
        except Exception as e:
            print(f"Error reading PDF: {e}")
            return False

    def clean_text(self):
        """Clean and normalize the extracted text"""
        # Replace multiple spaces with single space
        self.text = re.sub(r'\s+', ' ', self.text)
        return self.text

    def extract_personal_info(self):
        """Extract personal information"""
        personal_info = {}

        # Name
        name_match = re.search(r'My name is ([A-Za-z ]+), I was born', self.text)
        if name_match:
            personal_info['Name'] = name_match.group(1).strip()

        # Birth Date
        dob_match = re.search(r'was born on (\d{2}\.\d{2}\.\d{4})', self.text)
        if dob_match:
            dob = datetime.strptime(dob_match.group(1), '%d.%m.%Y')
            personal_info['Birth Date'] = dob.strftime('%Y-%m-%d 00:00:00')

            # Calculate age as of 2024
            age = 2024 - dob.year
            personal_info['Age'] = str(age)

        # Birth City and State
        city_state_match = re.search(r'born and raised in ([A-Za-z ]+), ([A-Za-z ]+)\.', self.text)
        if city_state_match:
            personal_info['Birth City'] = city_state_match.group(1).strip()
            personal_info['Birth State'] = city_state_match.group(2).strip()

        # Blood Group
        blood_group_match = re.search(r'My blood group is (A\+ve|A-ve|B\+ve|B-ve|AB\+ve|AB-ve|O\+ve|O-ve)', self.text)
        if blood_group_match:
            personal_info['Blood Group'] = blood_group_match.group(1)

        # Contact Information
        email_match = re.search(r'You can reach me at ([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})', self.text)
        if email_match:
            personal_info['Email'] = email_match.group(1)

        phone_match = re.search(r'or call me at (\+\d{1,3}\s\d{10})', self.text)
        if phone_match:
            personal_info['Contact Number'] = phone_match.group(1)

        # Nationality
        nationality_match = re.search(r'I am an ([A-Za-z]+) national', self.text)
        if nationality_match:
            personal_info['Nationality'] = nationality_match.group(1)

        return personal_info

    def extract_professional_info(self):
        """Extract professional information"""
        professional_info = {}

        # First professional role
        first_role_match = re.search(r'professional journey began on (July \d+, \d{4}).*?as a (Junior Developer).*?salary of ([\d,]+) INR', self.text)
        if first_role_match:
            join_date = datetime.strptime(first_role_match.group(1), '%B %d, %Y')
            professional_info['Joining Date of first professional role'] = join_date.strftime('%Y-%m-%d 00:00:00')
            professional_info['Designation of first professional role'] = first_role_match.group(2)
            professional_info['Salary of first professional role'] = first_role_match.group(3).replace(',', '')
            professional_info['Salary currency of first professional role'] = 'INR'

        # Current role
        current_role_match = re.search(r'current role at ([A-Za-z ]+ Analytics).*?on (June \d+, \d{4}).*?as a (Senior Data Engineer).*?earning ([\d,]+) INR', self.text)
        if current_role_match:
            professional_info['Current Organization'] = current_role_match.group(1)
            current_join_date = datetime.strptime(current_role_match.group(2), '%B %d, %Y')
            professional_info['Current Joining Date'] = current_join_date.strftime('%Y-%m-%d 00:00:00')
            professional_info['Current Designation'] = current_role_match.group(3)
            professional_info['Current Salary'] = current_role_match.group(4).replace(',', '')
            professional_info['Current Salary Currency'] = 'INR'

        # Previous organization
        prev_org_match = re.search(r'worked at ([A-Za-z ]+ Solutions?) from (February \d+, \d{4}) to (\d{4})', self.text)
        if prev_org_match:
            professional_info['Previous Organization'] = prev_org_match.group(1)
            prev_join_date = datetime.strptime(prev_org_match.group(2), '%B %d, %Y')
            professional_info['Previous Joining Date'] = prev_join_date.strftime('%Y-%m-%d 00:00:00')
            professional_info['Previous end year'] = prev_org_match.group(3)

        # Previous designation
        prev_designation_match = re.search(r'starting as a (Data Analyst).*?promotion in (\d{4})', self.text)
        if prev_designation_match:
            professional_info['Previous Starting Designation'] = prev_designation_match.group(1)

        return professional_info

    def extract_education_info(self):
        """Extract education information"""
        education_info = {}

        # High School
        school_match = re.search(r'high school education at ([A-Za-z\'. ]+ School), ([A-Za-z]+)', self.text)
        if school_match:
            education_info['High School'] = school_match.group(1) + ', ' + school_match.group(2)

        # 12th standard
        twelfth_match = re.search(r'12th standard in (\d{4}).*?(\d+\.\d+)% overall score', self.text)
        if twelfth_match:
            education_info['12th standard pass out year'] = twelfth_match.group(1)
            education_info['12th overall board score'] = str(float(twelfth_match.group(2)) / 100)

        # Undergraduate
        ug_match = re.search(r'B\.Tech in Computer Science.*?IIT Delhi.*?graduating.*?in (\d{4}).*?CGPA of (\d+\.\d+)', self.text)
        if ug_match:
            education_info['Undergraduate degree'] = 'B.Tech (Computer Science)'
            education_info['Undergraduate college'] = 'IIT Delhi'
            education_info['Undergraduate year'] = ug_match.group(1)
            education_info['Undergraduate CGPA'] = ug_match.group(2)

        # Graduation
        pg_match = re.search(r'M\.Tech in Data Science.*?IIT Bombay.*?in (\d{4}).*?CGPA of (\d+\.\d+)', self.text)
        if pg_match:
            education_info['Graduation degree'] = 'M.Tech (Data Science)'
            education_info['Graduation college'] = 'IIT Bombay'
            education_info['Graduation year'] = pg_match.group(1)
            education_info['Graduation CGPA'] = pg_match.group(2)

        return education_info

    def extract_certifications(self):
        """Extract certification information"""
        certifications = {}

        # Find all certification mentions
        cert_patterns = [
            (r'AWS Solutions Architect exam in (\d{4}) with a score of (\d+)', 'AWS Solutions Architect'),
            (r'Azure Data Engineer certification in (\d{4}) with (\d+) points', 'Azure Data Engineer'),
            (r'Project Management Professional certification.*?obtained in (\d{4})', 'Project Management Professional certification'),
            (r'SAFe Agilist certification.*?(\d+)% score', 'SAFe Agilist certification')
        ]

        for i, (pattern, cert_name) in enumerate(cert_patterns, 1):
            match = re.search(pattern, self.text)
            if match:
                certifications[f'Certifications {i}'] = cert_name

        return certifications

    def extract_technical_proficiency(self):
        """Extract technical proficiency information"""
        tech_info = {}

        # Technical proficiency section
        tech_section_match = re.search(r'technical proficiency.*?(SQL expertise.*?establishing him as an expert in the field)', self.text, re.DOTALL)
        if tech_section_match:
            tech_info['Technical Proficiency'] = tech_section_match.group(1).strip()

        return tech_info

    def generate_comments(self, key, value):
        """Generate comments based on the key and context from text"""
        comments_map = {
            'Birth City': 'Born and raised in the Pink City of India, his birthplace provides valuable regional profiling context',
            'Birth State': 'Born and raised in the Pink City of India, his birthplace provides valuable regional profiling context',
            'Age': 'As on year 2024. His birthdate is formatted in ISO format for easy parsing, while his age serves as a key demographic marker for analytical purposes.',
            'Blood Group': 'Emergency contact purposes.',
            'Nationality': 'Citizenship status is important for understanding his work authorization and visa requirements across different employment opportunities.',
            'Current Salary': 'This salary progression from his starting compensation to his current peak salary of 2,800,000 INR represents a substantial eight- fold increase over his twelve-year career span.',
            'Previous Starting Designation': 'Promoted in 2019',
            '12th standard pass out year': 'His core subjects included Mathematics, Physics, Chemistry, and Computer Science, demonstrating his early aptitude for technical disciplines.',
            '12th overall board score': 'Outstanding achievement',
            'Undergraduate year': 'Graduating with honors and ranking 15th among 120 students in his class.',
            'Undergraduate CGPA': 'On a 10-point scale',
            'Graduation college': 'Continued academic excellence at IIT Bombay',
            'Graduation CGPA': 'Considered exceptional and scoring 95 out of 100 for his final year thesis project.',
            'Certifications 1': "Vijay's commitment to continuous learning is evident through his impressive certification scores. He passed the AWS Solutions Architect exam in 2019 with a score of 920 out of 1000",
            'Certifications 2': 'Pursued in the year 2020 with 875 points.',
            'Certifications 3': 'Obtained in 2021, was achieved with an "Above Target" rating from PMI, These certifications complement his practical experience and demonstrate his expertise across multiple technology platforms.',
            'Certifications 4': 'Earned him an outstanding 98% score. Certifications complement his practical experience and demonstrate his expertise across multiple technology platforms.',
            'Technical Proficiency': 'In terms of technical proficiency, Vijay rates himself highly across various skills, with SQL expertise at a perfect 10 out of 10, reflecting his daily usage since 2012. His Python proficiency scores 9 out of 10, backed by over seven years of practical experience, while his machine learning capabilities rate 8 out of 10, representing five years of hands-on implementation. His cloud platform expertise, including AWS and Azure certifications, also rates 9 out of 10 with more than four years of experience, and his data visualization skills in Power BI and Tableau score 8 out of 10, establishing him as an expert in the field.'
        }

        return comments_map.get(key, "")

    def process_document(self):
        """Main method to process the document and extract all data"""
        print("Extracting text from PDF...")
        if not self.extract_text_from_pdf():
            return False

        print("Cleaning text...")
        self.clean_text()

        print("Extracting data from various sections...")
        # Extract data from all sections
        all_data = {}
        all_data.update(self.extract_personal_info())
        all_data.update(self.extract_professional_info())
        all_data.update(self.extract_education_info())
        all_data.update(self.extract_certifications())
        all_data.update(self.extract_technical_proficiency())

        # Convert to list of dictionaries for DataFrame
        records = []
        for i, (key, value) in enumerate(all_data.items(), 1):
            comment = self.generate_comments(key, value)
            records.append({
                '#': i,
                'Key': key,
                'Value': value,
                'Comments': comment
            })

        self.extracted_data = records
        return True

    def save_to_excel(self, output_path):
        """Save extracted data to Excel file"""
        if not self.extracted_data:
            print("No data to save. Please process the document first.")
            return False

        try:
            df = pd.DataFrame(self.extracted_data)

            # Reorder columns to match expected output
            df = df[['#', 'Key', 'Value', 'Comments']]

            # Save to Excel
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='Output', index=False)

                # Get the workbook and worksheet
                workbook = writer.book
                worksheet = writer.sheets['Output']

                # Adjust column widths
                worksheet.column_dimensions['A'].width = 5
                worksheet.column_dimensions['B'].width = 35
                worksheet.column_dimensions['C'].width = 25
                worksheet.column_dimensions['D'].width = 80

            print(f"Data successfully saved to {output_path}")
            return True

        except Exception as e:
            print(f"Error saving to Excel: {e}")
            return False

    def print_summary(self):
        """Print summary of extracted data"""
        print(f"\nExtracted {len(self.extracted_data)} records:")
        for record in self.extracted_data:
            print(f"{record['#']}. {record['Key']}: {record['Value']}")

In [19]:
# Instantiate the DocumentProcessor with the PDF file name
# Make sure 'Data Input.pdf' is uploaded to your Colab environment.
pdf_processor = DocumentProcessor("Data Input.pdf")

# Process the document
if pdf_processor.process_document():
    # Define output path for Excel file
    output_excel_path = 'extracted_data.xlsx'

    # Save extracted data to Excel
    pdf_processor.save_to_excel(output_excel_path)

    # Print summary of extracted data
    pdf_processor.print_summary()
else:
    print("Document processing failed.")


Extracting text from PDF...
Cleaning text...
Extracting data from various sections...
Data successfully saved to extracted_data.xlsx

Extracted 22 records:
1. Joining Date of first professional role: 2012-07-01 00:00:00
2. Designation of first professional role: Junior Developer
3. Salary of first professional role: 350000
4. Salary currency of first professional role: INR
5. Current Organization: Resse Analytics
6. Current Joining Date: 2021-06-15 00:00:00
7. Current Designation: Senior Data Engineer
8. Current Salary: 2800000
9. Current Salary Currency: INR
10. Previous Starting Designation: Data Analyst
11. High School: St. Xavier's School, Jaipur
12. 12th standard pass out year: 2007
13. 12th overall board score: 0.925
14. Undergraduate degree: B.Tech (Computer Science)
15. Undergraduate college: IIT Delhi
16. Undergraduate year: 2011
17. Undergraduate CGPA: 8.7
18. Certifications 1: AWS Solutions Architect
19. Certifications 2: Azure Data Engineer
20. Certifications 3: Project Man

In [ ]:
from google.colab import drive
drive.mount('/content/drive')